In [ ]:
import random

In [ ]:
def SISSRs(instance, delta, epsilon, iter_max):
    s_0 = solution_construction()
    s_curr = s_0
    s_best = s_0
    while (terminal_condition_met() == False):
        s = ruin_and_recreate(s_curr)
        if cost(s)<cost(s_curr)*(1+delta):
            s_curr = local_search(s)
            if cost(s_curr) < cost(s_best):
                s_best = s_curr
        if iterations_without_improvement >= iter_max:
            s_curr = perturbate(s_curr)
        delta = delta * epsilon

    return s_best 

           

In [ ]:
def sweep_removal_operator(solution, sigma):
    c_drones = get_drone_customers(solution[0])
    nbSweept = random.randint(0,int(len(c_drones) * sigma))

    grad_seed = random.randint(0, 360)
    c_drones = sort_by_angular(c_drones, grad_seed)
    for c in c_drones:
        if (len(solution[1]) < nbSweept):
            remove_customer(solution, c)
            solution[1].append(c)

    return solution

In [ ]:
def random_drone_customer_removal(solution, sigma):
    c_drones = get_drone_customers(solution[0])
    q = random.randint(0,int(len(c_drones) * sigma))
    
    
    while (len(solution[1]) < q):
        customer_to_remove = random.choice(c_drones)

        c_drones.remove(customer_to_remove)
        remove_customer(solution, customer_to_remove)
        solution[1].append(customer_to_remove)

    return solution

In [ ]:
def string_removal(n, coord, solution, c_average_removed, L_max):# n is the number of customers

    l_s_max = max_string_length(solution[0], L_max)
    k_s = n_strings_to_remove(c_average_removed, l_s_max)
    
    c_seed = random.randint(1, n)

    R = []

    c_adj = sort_by_euclidean_distance(c_seed, coord)

    for c in c_adj:
        if len(R) < k_s:
            if any(c in truck_tour for truck_tour in solution[0][0]):
                if c not in solution[1]:
                    tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if c in truck_tour), -1)
                    if tour_index not in R:
                        l = removed_string_length(l_s_max, solution[0][0][tour_index])
                        
                        solution[0][0][tour_index], solution[1] = remove_string(solution[0][0][tour_index], l, c)
                        R.append(tour_index)
            elif any(c in drone_tour for drone_tour in solution[0][1]):
                if c not in solution[1]:
                    tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
                    solution[0][1][tour_index], solution[1] = remove_drone_customer(solution[0][1][tour_index], c)


    return solution